<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Módulo 1: Crear una Red Neuronal Recurrente en TensorFlow 2.0



## Paso 1: Importar las librerías necesarias

## Paso 2: Pre procesado de datos


### Configurar parámetros del dataset

### Carga del dataset de IMDB

### Cortar secuencias de texto de la misma longitud

### Configurar parámetros de la capa de Embedding

## Paso 3: Construir la Red Neuronal Recurrente

### Definir el modelo

### Añadir la capa de embedding

### Añadir la capa de LSTM

- unidades: 128
- función de activación: tanh

In [ ]:
#!pip install -U numpy==1.18.5 --user

### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid

### Compilar el modelo

### Entrenar el modelo

Epoch 1/10
196/196 [==============================] - 24s 120ms/step - loss: 0.4903 - accuracy: 0.7688
Epoch 2/10
196/196 [==============================] - 25s 128ms/step - loss: 0.2923 - accuracy: 0.8834
Epoch 3/10
196/196 [==============================] - 28s 143ms/step - loss: 0.2312 - accuracy: 0.9123
Epoch 4/10
196/196 [==============================] - 38s 191ms/step - loss: 0.1906 - accuracy: 0.9292
Epoch 5/10
196/196 [==============================] - 35s 178ms/step - loss: 0.1555 - accuracy: 0.9426
Epoch 6/10
 26/196 [==>...........................] - ETA: 28s - loss: 0.1120 - accuracy: 0.9606

### Evaluar el modelo

In [ ]:
#mejorar el accuracy de la red 1

### Otro Ejemplo: Predecir el precio de las acciones de Google

In [ ]:
dataset_train = pd.read_csv('Data/Google_Stock_Price_Train.csv')


## Generación de texto usando RNN

Cuando se trabaja con texto en un modelo de lenguaje natural (recordar que los tokens son palabras o caracteres), usando cualquier red podríamos modelar la probabilidad del siguiente token (palabra) para construir un modelo de generación de texto. 

Este modelo de lenguaje natural podría capturar la estructura estadística del texto completo. Entonces, podríamos entrenar una red neuronal para predecir el siguiente carácter, ó de manera similar, podríamos entrenar al modelo para que prediga la siguiente palabra, dada una secuencia de palabras. En este apartado implementaremos un modelo a nivel de caracter.


<img src="Figures/63.png" alt="Grayscale Image" width="600">

### Implementing in Tensorflow
#### The Dataset
Usaremos un conjunto de datos que contiene las obras de Shakespeare.

In [ ]:
import tensorflow as tf
import numpy as np

path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Data Pre-Processing

Antes del entrenamiento, necesitamos asignar cadenas a números, extraer secuencias parcialmente superpuestas y empaquetarlas en una matriz numpy 3D de forma (secuencias, maxlen, caracteres únicos). 

### Construyendo la RNN

La red que construiremos será una red con una sola capa LSTM seguida de una ,capa densa con una función de activación softmax.

### Compile and Train the model

Usaremos la función de pérdida `categorical_crossentropy` para entrenar el modelo.

### Train the model

## Generate text